<a href="https://colab.research.google.com/github/BeommoKoo-dev/recommender-system/blob/beommo/book_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keybert
!pip install POT

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=cced5f95057575cfabab2c620eb7aaf6868386d97cb324f6c4f99a914e2d0384
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8ab68ed4ce4dc9fae1299142cad36a37e971c5f1d19b2c3b27d5347d6ed2edde
 

In [3]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
import gensim
from gensim.models import KeyedVectors
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

from gensim.models import LdaModel
from gensim import corpora
from sklearn.manifold import TSNE
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import pandas as pd
import json
import csv
import spacy

In [ ]:
data = []
book_ids = []
book_summaries = []
synopsis = []
bookCount = 200
keywordCount = 15
kw_model = KeyBERT()
sp_model = spacy.load("en_core_web_sm")

with open('/content/sample_data/booksummaries.txt', 'r') as f:
    bookReader = csv.reader(f, dialect='excel-tab')
    count = 0
    for bookData in bookReader:
      if count >= bookCount :
        break
      if count % 10 == 0:
        print(f'Reading File.. ({count}/{bookCount})')

      data.append(bookData)
      count += 1

# convert data to pandas dataframe
books = pd.DataFrame.from_records(data, columns=['book_id', 'freebase_id', 'book_title', 'author', 'publication_date', 'genre', 'summary'])

Reading File.. (0/200)
Reading File.. (10/200)
Reading File.. (20/200)
Reading File.. (30/200)
Reading File.. (40/200)
Reading File.. (50/200)
Reading File.. (60/200)
Reading File.. (70/200)
Reading File.. (80/200)
Reading File.. (90/200)
Reading File.. (100/200)
Reading File.. (110/200)
Reading File.. (120/200)
Reading File.. (130/200)
Reading File.. (140/200)
Reading File.. (150/200)
Reading File.. (160/200)
Reading File.. (170/200)
Reading File.. (180/200)
Reading File.. (190/200)


In [ ]:
book_ids = list(books['book_id'])
bookSummary = list(books['summary'])

[' Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, \'Beasts of England\'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares himself leader. 

In [ ]:
count = 0

for book_idx in range(0, bookCount):
    if count >= bookCount:
        break
    if count % 10 == 0:
        print(f'Reading File.. ({count}/{bookCount})')

    keywords = kw_model.extract_keywords(bookSummary[book_idx], top_n=keywordCount, vectorizer=KeyphraseCountVectorizer())
    keyList = [keyword[0] for keyword in keywords]
    # extracting needless names
    filtered_keywords = []
    for keyword in keyList:
        doc = sp_model(keyword)
        is_name = any(token.ent_type_ == "PERSON" and token.ent_iob_ != '0' for token in doc)

        if not is_name:
            filtered_keywords.append(keyword)

    keyString = ', '.join(filtered_keywords)

    synopsis.append(keyString)

    count += 1

2023-10-06 02:49:40,068 - KeyphraseVectorizer - INFO - It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
INFO:KeyphraseVectorizer:It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.


Reading File.. (0/200)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Reading File.. (10/200)
Reading File.. (20/200)
Reading File.. (30/200)
Reading File.. (40/200)
Reading File.. (50/200)
Reading File.. (60/200)
Reading File.. (70/200)
Reading File.. (80/200)
Reading File.. (90/200)
Reading File.. (100/200)
Reading File.. (110/200)
Reading File.. (120/200)
Reading File.. (130/200)
Reading File.. (140/200)
Reading File.. (150/200)
Reading File.. (160/200)
Reading File.. (170/200)
Reading File.. (180/200)
Reading File.. (190/200)


In [ ]:
# Writing .csv
csvFilePath = "book_keyword_extraction_" + str(keywordCount) + ".csv"
with open(csvFilePath, 'w', newline='', encoding='utf-8') as csvFile:
    headers = ['BookId', "Keywords"]
    writer = csv.DictWriter(csvFile, fieldnames=headers)
    writer.writeheader()

    for idx in range(0, bookCount):
        writer.writerow({'BookId': book_ids[idx], 'Keywords': synopsis[idx]})

In [6]:
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin', binary=True)

KeyboardInterrupt: ignored

In [4]:
import csv
book_data = open("/content/sample_data/book_keyword_extraction_15.csv", "r")
movie_data = open("/content/sample_data/movie_keyword_extraction_15.csv", "r")

In [7]:
book_rdr = csv.reader(book_data)
movie_rdr = csv.reader(movie_data)

In [8]:
books = {book[0]:book[1] for book in book_rdr}
movies = {movie[0]:movie[1] for movie in movie_rdr}

# for book in book_rdr :
#   books.append(book)

# for movie in movie_rdr :
#   movies.append(movie)

In [9]:
distance = model.wmdistance(books['620'], movies['tt0105112'])

/usr/local/lib/python3.10/dist-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


0.3134488466090895


In [10]:
print(movies)

{'MovieID': 'Keywords', 'tt0105112': 'ryan, ulster, sean, officers, terrorists, guards, jack, rescue, captured, troopers, officer, donnell, ireland, bodyguard, ned, guard, cab, hostage, accomplice', 'tt1204975': 'archie, billy, paddy, ronnie, flatbush, boys, grandson, bed, jackson, funeral, party, bar, malibu, teens, nightclub, brooklyn, wedding, night, son', 'tt0243655': '', 'tt0040897': 'dobbs, gold, cody, fred, prospector, fortune, lottery, prospecting, mined, bandits, barton, bruce, curtin, barter, finds, sees', 'tt0126886': 'reese, tracy, teacher, liaison, junior, student, senior, students, candidate, delaney, mcallister, matthew, molly, witherspoon, campbell', 'tt0286716': 'bruce, hulk, nanomeds, ross, scientist, talbot, banner, regeneration, experiments, havoc, todd, mutation, rampage, foster, superhuman, genetic, lucas', 'tt0090605': 'ripley, alien, aliens, spacecraft, cat, survivors, stranded, unconscious, chamber, crew, presence, marine, explosion, survived, weaver, disorient

In [35]:
def recommend_movie(book_id, recommend_count) :
  if book_id not in books :
    print("there is no book")
    return
  book_keyword = books[book_id]
  if len(book_keyword) == 0 :
    print("there is no book summarization")
    return
  count = 0
  res = []

  for movie_id, movie_keyword in movies.items() :
    if len(movie_keyword) == 0 :
      continue
    distance = model.wmdistance(book_keyword, movie_keyword)
    res.append({'movie_id' : movie_id, 'distance' : distance})

  sorted_res = sorted(res, key=lambda x: x['distance'])
  # print(sorted_res)
  return sorted_res[:recommend_count]


In [36]:
res = recommend_movie('620', 2)
print(res)

[{'movie_id': 'tt0450259', 'distance': 0.154388136919629}, {'movie_id': 'tt0458339', 'distance': 0.17206654466430915}]


MovieID
tt0105112
tt1204975
tt0243655
tt0040897
tt0126886
tt0286716
tt0090605
tt0243155
tt0121765
tt0443453
tt0107131
tt0110364
tt0450259
tt0143145
tt0435625
tt0343818
tt0120815
tt0253754
tt1951264
tt0118849
tt0120783
tt0114814
tt0101700
tt0377092
tt0102768
tt1259521
tt2084970
tt0108399
tt0017136
tt2356777
tt0215750
tt0082096
tt0408985
tt0107822
tt1120985
tt0018455
tt0499448
tt0133152
tt1289401
tt2101441
tt0475784
tt1536537
tt0113957
tt0100507
tt0302886
tt0460681
tt3011894
tt0114319
tt3748528
tt0107207
tt0120363
tt0109707
tt0103893
tt1217209
tt0116126
tt0140352
tt0406375
tt0120794
tt3659388
tt0160127
tt0103772
tt0099810
tt0116705
tt0298148
tt0351283
tt0102744
tt1298650
tt1187064
tt0149261
tt0106977
tt0452594
tt2660888
tt0103064
tt0959337
tt0295700
tt0107048
tt0416320
tt0353969
tt0405159
tt1781769
tt0337741
tt0120828
tt0120903
tt0232500
tt0113442
tt0396171
tt0458352
tt1392190
tt0458339
tt0110322
tt0324216
tt0382625
tt0101545
tt0360486
tt0129290
tt1228705
tt1430132
tt0133093
tt0103241
tt